**need to be fixed**: repeated image got downloaded on every iteration!!!

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options=chrome_options)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 983 kB 5.2 MB/s 
     |████████████████████████████████| 138 kB 50.4 MB/s 
     |████████████████████████████████| 359 kB 35.9 MB/s 
     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 4.0 MB 34.7 MB/s 
     |████████████████████████████████| 58 kB 5.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 http://se

In [ ]:
from PIL import Image
import time
import requests
import os
import io
import hashlib

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)

    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)

        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")

        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [ ]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [ ]:
def search_and_download(search_term:str, number_images:int, chrome_options:str,target_path='./images'):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome('chromedriver',options=chrome_options) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)

    for elem in res:
        persist_image(target_folder,elem)

In [ ]:
# remarks: repeated image got downloaded on every iteration
search_and_download(
    search_term = 'metal waste',
    number_images = 36,
    chrome_options = chrome_options
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead


Found: 100 search results. Extracting links from 0:100


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead


Found: 36 image links, done!
SUCCESS - saved https://www.rubicon.com/wp-content/uploads/2021/06/ferrous-and-non-ferrous-metals.jpg - as ./images/metal_waste/3720c7d6c0.jpg
SUCCESS - saved http://admin.acceleratingscience.com/metals/wp-content/uploads/sites/4/2018/07/istock-147698251_scrapmetalcubes-1.jpg - as ./images/metal_waste/85d8c259c1.jpg
SUCCESS - saved https://www.belsonsteel.com/wp-content/uploads/2020/03/AdobeStock_264863308.jpg - as ./images/metal_waste/40b50a9a20.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTaWZcYlLu2wOEXznts3vmfVFPSmCy1vcZWHg&usqp=CAU - as ./images/metal_waste/d57bcbc046.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQIKy9qcN2ELJnFHcN3fIOQmuhffS2KcJ528g&usqp=CAU - as ./images/metal_waste/e96c51dc4f.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSHM-lAkzt3Po6baWdVuoQSbPECD-CUWyjE_g&usqp=CAU - as ./images/metal_waste/f3e5245b44.jpg
SUCCESS - saved https://www.jdogjunkremoval.c

In [ ]:
# download file from colab to local
!zip -r ./zipped_files/metal_waste.zip ./images/metal_waste

from google.colab import files
files.download('./zipped_files/metal_waste.zip')

  adding: images/metal_waste/ (stored 0%)
  adding: images/metal_waste/6a70e58699.jpg (deflated 0%)
  adding: images/metal_waste/e852ed2b46.jpg (deflated 0%)
  adding: images/metal_waste/9526e2a666.jpg (deflated 0%)
  adding: images/metal_waste/d57bcbc046.jpg (deflated 3%)
  adding: images/metal_waste/f3e5245b44.jpg (deflated 2%)
  adding: images/metal_waste/e8e612a146.jpg (deflated 2%)
  adding: images/metal_waste/7f3699a1dc.jpg (deflated 3%)
  adding: images/metal_waste/e96c51dc4f.jpg (deflated 1%)
  adding: images/metal_waste/5d32a25e74.jpg (deflated 2%)
  adding: images/metal_waste/ea0a7a40bf.jpg (deflated 1%)
  adding: images/metal_waste/fe42c537e2.jpg (deflated 2%)
  adding: images/metal_waste/85d8c259c1.jpg (deflated 0%)
  adding: images/metal_waste/abf4c2d671.jpg (deflated 2%)
  adding: images/metal_waste/3720c7d6c0.jpg (deflated 1%)
  adding: images/metal_waste/b05e2453c7.jpg (deflated 3%)
  adding: images/metal_waste/ce60828496.jpg (deflated 3%)
  adding: images/metal_waste/e

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>